<a href="https://colab.research.google.com/github/Tiabet/BaekJoon/blob/main/%EB%8C%80%EC%B6%9C%EB%93%B1%EA%B8%89%EC%98%88%EC%B8%A1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer

In [2]:
# 데이터 불러오기 및 전처리
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')
sample_submission = pd.read_csv('sample_submission.csv')

In [3]:
train_df

,ID,대출금액,대출기간,근로기간,주택소유상태,연간소득,부채_대비_소득_비율,총계좌수,대출목적,최근_2년간_연체_횟수,총상환원금,총상환이자,총연체금액,연체계좌수,대출등급
0,TRAIN_00000,12480000,36 months,6 years,RENT,72000000,18.90,15,부채 통합,0,0,0.0,0.0,0.0,C
1,TRAIN_00001,14400000,60 months,10+ years,MORTGAGE,130800000,22.33,21,주택 개선,0,373572,234060.0,0.0,0.0,B
2,TRAIN_00002,12000000,36 months,5 years,MORTGAGE,96000000,8.60,14,부채 통합,0,928644,151944.0,0.0,0.0,A
3,TRAIN_00003,14400000,36 months,8 years,MORTGAGE,132000000,15.09,15,부채 통합,0,325824,153108.0,0.0,0.0,C
4,TRAIN_00004,18000000,60 months,Unknown,RENT,71736000,25.39,19,주요 구매,0,228540,148956.0,0.0,0.0,B
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96289,TRAIN_96289,14400000,36 months,10+ years,MORTGAGE,210000000,9.33,33,신용 카드,0,974580,492168.0,0.0,0.0,C
96290,TRAIN_96290,28800000,60 months,10+ years,MORTGAGE,132000000,5.16,25,주택 개선,0,583728,855084.0,0.0,0.0,E
96291,TRAIN_96291,14400000,36 months,1 year,MORTGAGE,84000000,11.24,22,신용 카드,0,1489128,241236.0,0.0,0.0,A
96292,TRAIN_96292,15600000,36 months,5 years,MORTGAGE,66330000,17.30,21,부채 통합,2,1378368,818076.0,0.0,0.0,D


1. 대출 목적 비슷한 것들은 묶기
2. 근로기간 언노운 처리하고 1이하랑 1 묶기
3. 주택소유상태 ANY 처리
4. 연체계좌수, 총연체금액 드랍

In [4]:
train_df.drop(['연체계좌수', '총연체금액'], axis=1, inplace = True)

In [5]:
train_df.loc[train_df['주택소유상태']=='ANY','주택소유상태'] = 'MORTGAGE'

In [6]:
train_df['주택소유상태'].value_counts()

MORTGAGE    47935
RENT        37705
OWN         10654
Name: 주택소유상태, dtype: int64

In [7]:
train_df.loc[train_df['근로기간']=='3','근로기간']='3 years'
train_df.loc[train_df['근로기간']=='1 year','근로기간']='1 years'
train_df.loc[train_df['근로기간']=='<1 year','근로기간']='< 1 year'
train_df.loc[train_df['근로기간']=='10+years','근로기간']='10+ years'
train_df.value_counts('근로기간')

근로기간
10+ years    32481
2 years       8450
< 1 year      8144
3 years       7670
1 years       6305
Unknown       5671
5 years       5665
4 years       5588
8 years       4888
6 years       3874
7 years       3814
9 years       3744
dtype: int64

In [8]:
values_map=dict({'6 years':5,
 '10+ years':10,
 '5 years':1,
 '8 years':5,
 '9 years':5,
 '2 years':1,
 '1 years':1,
 '3 years':1,
 '7 years':5,
 '4 years':5,
 '< 1 year':1})

In [9]:
train_df.loc[train_df['근로기간'] == 'Unknown', '근로기간'] = np.random.choice([1,5,10], size=sum(train_df['근로기간'] == 'Unknown'))

In [10]:
train_df['근로기간'] = train_df['근로기간'].replace(values_map)

In [11]:
train_df.loc[train_df['근로기간']==6,'근로기간']= np.random.randint(1,3)

In [12]:
train_df['근로기간'].value_counts()

1     38097
10    34397
5     23800
Name: 근로기간, dtype: int64

In [13]:
train_df['대출목적'].value_counts()

부채 통합     55150
신용 카드     24500
주택 개선      6160
기타         4725
주요 구매      1803
의료         1039
자동차         797
소규모 사업      787
이사          506
휴가          466
주택          301
재생 에너지       60
Name: 대출목적, dtype: int64

In [14]:
train_df.loc[~train_df['대출목적'].isin(['부채 통합', '신용 카드', '기타']), '대출목적'] = '기타'

In [15]:
train_df['대출목적'].value_counts()

부채 통합    55150
신용 카드    24500
기타       16644
Name: 대출목적, dtype: int64

In [16]:
train_df.loc[train_df['대출기간']==' 36 months', '대출기간'] = 36
train_df.loc[train_df['대출기간']==' 60 months', '대출기간'] = 60

In [17]:
train_df.value_counts('대출기간')

대출기간
36    64479
60    31815
dtype: int64

In [18]:
train_df['최근_2년간_연체_횟수'] = np.where(train_df['최근_2년간_연체_횟수'] > 2, 2, train_df['최근_2년간_연체_횟수'])

In [19]:
train_df['대출기간'] = train_df['대출기간'].astype(int)

In [20]:
test_df.drop(['연체계좌수', '총연체금액'], axis=1, inplace=True)
test_df.loc[test_df['주택소유상태']=='ANY','주택소유상태'] = 'MORTGAGE'
test_df.loc[test_df['근로기간'] == '3', '근로기간'] = '3 years'
test_df.loc[test_df['근로기간'] == '1 year', '근로기간'] = '1 years'
test_df.loc[test_df['근로기간'] == '<1 year', '근로기간'] = '< 1 year'
test_df.loc[test_df['근로기간'] == '10+years', '근로기간'] = '10+ years'
test_df.loc[test_df['근로기간'] == 'Unknown', '근로기간'] = np.random.choice([1,5,10], size=sum(test_df['근로기간'] == 'Unknown'))
test_df['근로기간'] = test_df['근로기간'].replace(values_map)
test_df.loc[test_df['근로기간'] == 6, '근로기간'] = np.random.randint(1, 3)
test_df.loc[~test_df['대출목적'].isin(['부채 통합', '신용 카드', '기타']), '대출목적'] = '기타'
test_df.loc[test_df['대출기간']==' 36 months', '대출기간'] = 36
test_df.loc[test_df['대출기간']==' 60 months', '대출기간'] = 60
test_df['최근_2년간_연체_횟수'] = np.where(test_df['최근_2년간_연체_횟수'] > 2, 2, test_df['최근_2년간_연체_횟수'])
test_df['대출기간'] = test_df['대출기간'].astype(int)

In [21]:
train_data = train_df.copy()
test_data = test_df.copy()

In [22]:
train_data.drop(columns=['ID'], inplace=True)
test_data.drop(columns=['ID'], inplace=True)

In [23]:
train_y =train_data['대출등급']
train_X = train_data[train_data.columns.difference(['대출등급'])]
test_X = test_data[test_data.columns.difference(['대출등급'])]

In [24]:
train_X = pd.get_dummies(train_X)
test_X = pd.get_dummies(test_X)

전처리 완료, 모델 적용

In [25]:
train_X

,근로기간,대출금액,대출기간,부채_대비_소득_비율,연간소득,총계좌수,총상환원금,총상환이자,최근_2년간_연체_횟수,대출목적_기타,대출목적_부채 통합,대출목적_신용 카드,주택소유상태_MORTGAGE,주택소유상태_OWN,주택소유상태_RENT
0,5,12480000,36,18.90,72000000,15,0,0.0,0,0,1,0,0,0,1
1,10,14400000,60,22.33,130800000,21,373572,234060.0,0,1,0,0,1,0,0
2,1,12000000,36,8.60,96000000,14,928644,151944.0,0,0,1,0,1,0,0
3,5,14400000,36,15.09,132000000,15,325824,153108.0,0,0,1,0,1,0,0
4,1,18000000,60,25.39,71736000,19,228540,148956.0,0,1,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96289,10,14400000,36,9.33,210000000,33,974580,492168.0,0,0,0,1,1,0,0
96290,10,28800000,60,5.16,132000000,25,583728,855084.0,0,1,0,0,1,0,0
96291,1,14400000,36,11.24,84000000,22,1489128,241236.0,0,0,0,1,1,0,0
96292,1,15600000,36,17.30,66330000,21,1378368,818076.0,2,0,1,0,1,0,0


In [26]:
test_X

,근로기간,대출금액,대출기간,부채_대비_소득_비율,연간소득,총계좌수,총상환원금,총상환이자,최근_2년간_연체_횟수,대출목적_기타,대출목적_부채 통합,대출목적_신용 카드,주택소유상태_MORTGAGE,주택소유상태_OWN,주택소유상태_RENT
0,5,16800000,36,19.64,132000000,12,394692,146604.0,0,1,0,0,1,0,0
1,1,8400000,36,15.84,89971200,25,0,0.0,0,0,1,0,0,0,1
2,5,17280000,36,8.41,150000000,20,1786980,281820.0,0,0,0,1,0,0,1
3,1,14400000,36,13.72,66000000,30,669024,281724.0,1,0,0,1,1,0,0
4,1,27600000,36,30.50,55200000,12,1250052,614844.0,0,0,0,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
64192,1,30000000,36,22.08,78000000,27,1307532,763380.0,2,0,1,0,1,0,0
64193,10,30000000,60,12.06,109200000,26,960612,1245252.0,0,0,1,0,1,0,0
64194,10,6120000,36,28.80,39600000,33,131520,80880.0,0,0,1,0,0,0,1
64195,10,11520000,36,25.44,66000000,41,1339536,601872.0,1,0,1,0,1,0,0


방법 1. lgbm 적용 + 하이퍼파라미터 튜닝
방법 2. lgbm 적용한 다음 스태킹 앙상블
방법 3. cat, lgbm, rf 등등 소프트앙상블

In [27]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(train_X, train_y, test_size=0.2, random_state=42)

In [28]:
import lightgbm as lgbm

lgbm_params = {
          'objective': 'multiclass',
          'num_class': 6,
          'metric': 'multi_logloss',
          'boosting_type': 'gbdt',
          'n_estimators': 15000,
          'num_leaves': 8,
          'min_child_samples': 12,
          'learning_rate': 0.15,
          'max_depth': 10,
          'subsample': 0.9,
          'colsample_bytree': 0.9,
          'subsample_freq': 1,
          'reg_alpha': 0.1,
          'reg_lambda': 0.8,
          'verbose': -1}
lgbm_model = lgbm.LGBMClassifier(**lgbm_params)

In [29]:
# Fit the model to the training data
#lgbm_model.fit(train_X, train_y)

# Make predictions on the test data
#predictions = lgbm_model.predict(test_X)

In [30]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [31]:
sub = pd.read_csv('sample_submission.csv')
predictions = np.zeros(len(sub))
sub['대출등급'] = predictions
sub.to_csv('drive/MyDrive/submission.csv', index=False)

하이퍼파라미터 튜닝

In [32]:
pip install optuna

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 413.4/413.4 kB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.4/233.4 kB 19.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.7/78.7 kB 8.8 MB/s eta 0:00:00


In [ ]:
import lightgbm as lgbm
import optuna
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import f1_score, log_loss

X, y = train_X.sample(20000,random_state = 42), train_y.sample(20000,random_state = 42)

def macro_f1_score(y_true, preds):
    preds = preds.argmax(axis=1)  # Convert probability predictions to class labels
    return 'macro_f1', f1_score(y_true, preds, average='macro'), True

def objective(trial):
    params = {
        'objective': 'multiclass',
        'num_class': 6,
        'boosting_type': 'gbdt',
        'n_estimators': trial.suggest_int('n_estimators', 100, 20000),
        'num_leaves': trial.suggest_int('num_leaves', 2, 50),
        'min_child_samples': trial.suggest_int('min_child_samples', 5, 100),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.5, log = True),
        'max_depth': trial.suggest_int('max_depth', 3, 15),
        'subsample': trial.suggest_float('subsample', 0.6, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.6, 1.0),
        'subsample_freq': trial.suggest_int('subsample_freq', 1, 10),
        'reg_alpha': trial.suggest_float('reg_alpha', 1e-3, 10.0, log = True),
        'reg_lambda': trial.suggest_float('reg_lambda', 1e-3, 10.0, log = True),
        'verbose': -1
    }

    loglosses = []
    skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    for train_index, valid_index in skf.split(X, y):
        X_train, X_valid = X.iloc[train_index], X.iloc[valid_index]
        y_train, y_valid = y.iloc[train_index], y.iloc[valid_index]

        model = lgbm.LGBMClassifier(**params)
        model.fit(X_train, y_train, eval_set=[(X_valid, y_valid)], eval_metric='multi_logloss')
        preds_proba = model.predict_proba(X_valid)
        logloss = log_loss(y_valid, preds_proba)
        loglosses.append(logloss)

    print('1번끝')
    return np.mean(loglosses)


study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=100)

best_params = study.best_params


[I 2024-02-01 09:20:33,943] A new study created in memory with name: no-name-42a4bbe1-4bec-48d9-90c0-3b1d291a1b2c


In [ ]:
best_params = pd.DataFrame(best_params, index=[0])
best_params.to_csv('drive/MyDrive/best_params.csv')

In [ ]:
lgbm_model = lgbm.LGBMClassifier(**best_params)
# Fit the model to the training data
lgbm_model.fit(train_X, train_y)

# Make predictions on the test data
predictions = lgbm_model.predict(test_X)

sub = pd.read_csv('sample_submission.csv')
predictions = np.zeros(len(sub))
sub['대출등급'] = predictions
sub.to_csv('drive/MyDrive/submission_lgbmtuning.csv', index=False)